##  Summary some properties of NATS dataset

### Dataset

There are 3 dataset, cifar10, cifar100, imagenet16-120, each was resplit into
train, dev, test by  the author. Specifically:
1.    cifar10: 25, 25, 10
2.    cifar100: 50, 5, 5
3.    imagenet16-120: 151.7, 3, 3

**You can query the train/test acc, loss on every epochs.**

**You can get the weights of trained architecture**

*The performance of architectures with a fix FLOPs can vary in it val acc, so our natural desirable architecture is the one that has the highest performance given a FLOPs*

*What exciting is that, there is architecures with the same or even better performance than Resnet but have much lower FLOPs for you to search!*

### Metrics

1. Computational metrics include FLOPs, number of parameters, latency. You also have the training and evaluation time as well, at every epochs.
2. 

**tss search space**
- conv is abbreviate for a sequence of relu, conv, batchnorm
- each stack in the macro structure has 5 identical cell. Stacks are identical in topology too, different in its channel number.
- There are 2 set of hyperparameters for this search space. H0 and H1. They are just the same except that H0 train with 12 epochs and H1 train with 200 epochs.

**sss search space**

There are 12 epochs and 90 epochs result data.

**Comments**

- Pyramic structure gives near optima pareto front w.r.t FLOPs and acc on the tss.
- When we directly transfer the best architecture from one dataset to the other, no guarantee of performance. Which means we need a more transferable NAS algorithm.
- different training runs of the same architecture can affect the overal ranking, because the randomess in weight initialization, data augmentation,... but we do not want this ranking to be affected too much.
- There are 13 sota search method code available on the benchmark dataset. So good for research. These methods are cited in the paper.
- There is one thing tobe considered, even random search on NATS search space only behind the best search algorithm 1-2% on test acc. So the benchmark is actually not the final destination to test your algorithm. If your algorithm can perform much better than the random search on a specific search space that you have thing to talk about.
- the paper recommended REA for the searching algorithm if you have enough computation. But I dout it effectiveness, since the performance distance between it and random search is not that far. ~1%

**Avoid overfitting on the dataset**
- use the same hyperparameter of the searching algorithm for different dataset and search space.
- No regularization for specific operation, not sure what is this means
- Use the provided performance, do not attempt to retrain!
- report results of multiple searching runs, report mean and std.
- 
**Experiments Ideas**
- I am very much want to compare the effectiveness of training free algorithm to those algorithm in the paper, see which search alg can find better architecture!

In [1]:
from pprint import pprint
from nats_bench import NATStopology
from nats_bench import create
import xautodl

In [2]:
api = create(r'C:\Users\ASUS\Desktop\nas\nas-without-training\NATS-tss-v1_0-3ffb9-simple', 'tss', True)

[2021-10-09 09:12:51] Try to create the NATS-Bench (topology) api from C:\Users\ASUS\Desktop\nas\nas-without-training\NATS-tss-v1_0-3ffb9-simple with fast_mode=True
[2021-10-09 09:12:51] Create NATS-Bench (topology) done with 0/15625 architectures avaliable.


In [3]:
# you can get the unique string of the architecture by its index
api.get_unique_str(1)

[2021-10-09 09:12:53] Call query_index_by_arch with arch=1


'((0)@nor_conv_3x3)@avg_pool_3x3+(0)@nor_conv_3x3+((0)@nor_conv_3x3)@nor_conv_3x3+0'

In [12]:
# hp is hyperparameter indicator, could be 12 or 200, which mean the number of training epochs
pprint(api.query_info_str_by_arch(1))

[2021-10-08 07:16:48] Call query_info_str_by_arch with arch=1and hp=12
[2021-10-08 07:16:48] Call query_index_by_arch with arch=1
[2021-10-08 07:16:48] Call _prepare_info with index=1 skip because it is in arch2infos_dict
('|nor_conv_3x3~0|+|nor_conv_3x3~0|avg_pool_3x3~1|+|skip_connect~0|nor_conv_3x3~1|skip_connect~2|\n'
 "datasets : ['cifar10-valid', 'cifar10', 'cifar100', 'ImageNet16-120'], "
 'extra-info : arch-index=1\n'
 'cifar10-valid  FLOP=113.95 M, Params=0.802 MB, latency=16.85 ms.\n'
 'cifar10-valid  train : [loss = 0.382 & top1 = 86.97%], valid : [loss = 0.514 '
 '& top1 = 82.83%]\n'
 'cifar10        FLOP=113.95 M, Params=0.802 MB, latency=16.85 ms.\n'
 'cifar10        train : [loss = 0.243 & top1 = 91.69%], test  : [loss = 0.362 '
 '& top1 = 88.22%]\n'
 'cifar100       FLOP=113.96 M, Params=0.808 MB, latency=15.36 ms.\n'
 'cifar100       train : [loss = 1.271 & top1 = 63.76%], valid : [loss = 1.495 '
 '& top1 = 57.80%], test : [loss = 1.478 & top1 = 58.26%]\n'
 'ImageNet16-

In [14]:
api.get_more_info(1, 'cifar10', hp = 200)

[2021-10-08 07:22:06] Call the get_more_info function with index=1, dataset=cifar10, iepoch=None, hp=200, and is_random=True.
[2021-10-08 07:22:06] Call query_index_by_arch with arch=1
[2021-10-08 07:22:06] Call _prepare_info with index=1 skip because it is in arch2infos_dict


{'train-loss': 0.002528535761758685,
 'train-accuracy': 99.97,
 'train-per-time': 21.69641375541687,
 'train-all-time': 4339.282751083374,
 'comment': 'In this dict, train-loss/accuracy/time is the metric on the train+valid sets of CIFAR-10. The test-loss/accuracy/time is the performance of the CIFAR-10 test set after training on the train+valid sets by 200 epochs. The per-time and total-time indicate the per epoch and total time costs, respectively.',
 'test-loss': 0.305450784778595,
 'test-accuracy': 93.75,
 'test-per-time': 1.5016795907701752,
 'test-all-time': 300.335918154035}

In [4]:
config = api.get_net_config(12, 'cifar10')
print(config)

[2021-10-09 09:12:58] Call the get_net_config function with index=12, dataset=cifar10.
[2021-10-09 09:12:58] Call clear_params with archive_root=C:\Users\ASUS\Desktop\nas\nas-without-training\NATS-tss-v1_0-3ffb9-simple and index=12
{'name': 'infer.tiny', 'C': 16, 'N': 5, 'arch_str': '|none~0|+|none~0|none~1|+|none~0|nor_conv_3x3~1|avg_pool_3x3~2|', 'num_classes': 10}


In [6]:
from xautodl.models import get_cell_based_tiny_net


In [7]:
network = get_cell_based_tiny_net(config)


In [11]:
pprint(network)

TinyNetwork(
  TinyNetwork(C=16, N=5, L=17)
  (stem): Sequential(
    (0): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (cells): ModuleList(
    (0): InferCell(
      info :: nodes=4, inC=16, outC=16, [1<-(I0-L0) | 2<-(I0-L1,I1-L2) | 3<-(I0-L3,I1-L4,I2-L5)], |none~0|+|none~0|none~1|+|none~0|nor_conv_3x3~1|avg_pool_3x3~2|
      (layers): ModuleList(
        (0): Zero(C_in=16, C_out=16, stride=1)
        (1): Zero(C_in=16, C_out=16, stride=1)
        (2): Zero(C_in=16, C_out=16, stride=1)
        (3): Zero(C_in=16, C_out=16, stride=1)
        (4): ReLUConvBN(
          (op): Sequential(
            (0): ReLU()
            (1): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
            (2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          )
        )
        (5): POOLING(
          (op): AvgPo

In [12]:
from torchsummary import summary


In [13]:
summary(network, (3,32,32))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 16, 32, 32]             432
       BatchNorm2d-2           [-1, 16, 32, 32]              32
              Zero-3           [-1, 16, 32, 32]               0
              Zero-4           [-1, 16, 32, 32]               0
              Zero-5           [-1, 16, 32, 32]               0
              Zero-6           [-1, 16, 32, 32]               0
              ReLU-7           [-1, 16, 32, 32]               0
            Conv2d-8           [-1, 16, 32, 32]           2,304
       BatchNorm2d-9           [-1, 16, 32, 32]              32
       ReLUConvBN-10           [-1, 16, 32, 32]               0
        AvgPool2d-11           [-1, 16, 32, 32]               0
          POOLING-12           [-1, 16, 32, 32]               0
        InferCell-13           [-1, 16, 32, 32]               0
             Zero-14           [-1, 16,

In [18]:
import inspect


NameError: name 'inpsect' is not defined

In [21]:
inspect.getmembers(api, inspect.ismethod)

[('__getitem__',
  <bound method NASBenchMetaAPI.__getitem__ of NATStopology(1/15625 architectures, fast_mode=True, file=NATS-tss-v1_0-3ffb9-simple)>),
 ('__init__',
  <bound method NATStopology.__init__ of NATStopology(1/15625 architectures, fast_mode=True, file=NATS-tss-v1_0-3ffb9-simple)>),
 ('__len__',
  <bound method NASBenchMetaAPI.__len__ of NATStopology(1/15625 architectures, fast_mode=True, file=NATS-tss-v1_0-3ffb9-simple)>),
 ('__repr__',
  <bound method NASBenchMetaAPI.__repr__ of NATStopology(1/15625 architectures, fast_mode=True, file=NATS-tss-v1_0-3ffb9-simple)>),
 ('_prepare_info',
  <bound method NASBenchMetaAPI._prepare_info of NATStopology(1/15625 architectures, fast_mode=True, file=NATS-tss-v1_0-3ffb9-simple)>),
 ('_query_info_str_by_arch',
  <bound method NASBenchMetaAPI._query_info_str_by_arch of NATStopology(1/15625 architectures, fast_mode=True, file=NATS-tss-v1_0-3ffb9-simple)>),
 ('_show',
  <bound method NASBenchMetaAPI._show of NATStopology(1/15625 architectu